In [1]:
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
import scipy
import matplotlib
import tqdm
from scipy import integrate, fft, sparse
import pandas as pd
import seaborn as sns
import random
import anndata
import os
from sklearn.metrics import confusion_matrix
import loompy as lp

import anndata
import scanpy as sc

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import kneighbors_graph

import igraph as ig
import leidenalg as la


In [2]:
import sys
import importlib
sys.path.insert(0, '/home/tchari/monod/src/')

In [3]:
import monod
#importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, inference, analysis, mminference

In [425]:
import warnings
warnings.filterwarnings('ignore', category=PendingDeprecationWarning) 

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [333]:
#From https://github.com/pachterlab/CGCCP_2023/blob/main/Code/Preprint/Preprocess_Data.ipynb, B08 sample

#Already selected for hvgs
ds = lp.connect('/home/tchari/counts/allen_bivi/loom/B08_processed_hv.loom')
ds

,,,,,,,,Cell Type,L5 IT,L2/3 IT,L5 IT,L2/3 IT,L6 CT,L5 IT,L5 IT,L6 IT,L2/3 IT,L2/3 IT,...
,,,,,,,,barcode,AAACCCAAGTGAACAT,AAACCCAAGTTGAATG,AAACCCACACGATAGG,AAACCCACAGAGGGTT,AAACCCAGTACTAAGA,AAACCCAGTACTGGGA,AAACCCAGTCGTATGT,AAACCCATCATACAGC,AAACCCATCATAGGCT,AAACGAAAGAAGCGAA,...
,,,,,,,,class_label,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,Glutamatergic,...
,,,,,,,,cluster_id,42,40,42,40,60,42,42,45,40,40,...
,,,,,,,,cluster_label,L5 IT Tcap_2,L2/3 IT,L5 IT Tcap_2,L2/3 IT,L6 CT Cpa6_1,L5 IT Tcap_2,L5 IT Tcap_2,L6 IT Sulf1_1,L2/3 IT,L2/3 IT,...
,,,,,,,,obs_names,1,2,5,6,7,8,9,10,11,13,...
,,,,,,,,subclass_label,L5 IT,L2/3 IT,L5 IT,L2/3 IT,L6 CT,L5 IT,L5 IT,L6 IT,L2/3 IT,L2/3 IT,...
Spliced,dispersions,dispersions_norm,gene_id,gene_name,highly_variable,means,var_names,,,,,,,,,,,,...
0,0.9820677777884145,2.030395746231079,ENSMUSG00000063558.4,Aox1,1,0.004415014219732092,80-u,,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...
0,1.8735677492433074,4.441442012786865,ENSMUSG00000010175.13,Prox1,1,0.3310880533908806,124-u,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
0,1.0491018338864364,2.1265413761138916,ENSMUSG00000015355.13,Cd48,1,0.002022428262812959,135-u,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...


In [400]:
meta[meta['cell_type'].isin(['L2/3 IT','L6 CT'])]

,cell_type,cell_barcode,cluster_id,class_label,sub_label,cell_type_id
1,L2/3 IT,AAACCCAAGTTGAATG,40,Glutamatergic,L2/3 IT,2
3,L2/3 IT,AAACCCACAGAGGGTT,40,Glutamatergic,L2/3 IT,2
4,L6 CT,AAACCCAGTACTAAGA,60,Glutamatergic,L6 CT Cpa6_1,6
8,L2/3 IT,AAACCCATCATAGGCT,40,Glutamatergic,L2/3 IT,2
9,L2/3 IT,AAACGAAAGAAGCGAA,40,Glutamatergic,L2/3 IT,2
...,...,...,...,...,...,...
6400,L6 CT,TTTGGAGCAAGAGGTC,60,Glutamatergic,L6 CT Cpa6_1,6
6401,L6 CT,TTTGGAGCACAGCATT,60,Glutamatergic,L6 CT Cpa6_1,6
6402,L2/3 IT,TTTGGAGCACCTGTCT,40,Glutamatergic,L2/3 IT,2
6413,L2/3 IT,TTTGTTGAGGGCCAAT,40,Glutamatergic,L2/3 IT,2


In [401]:
meta[meta['cell_type'].isin(['Vip'])]

,cell_type,cell_barcode,cluster_id,class_label,sub_label,cell_type_id
56,Vip,AAAGGTATCGCATTGA,20,GABAergic,Vip Gpc3,19
63,Vip,AAAGTCCGTCCAAATC,20,GABAergic,Vip Gpc3,19
77,Vip,AAATGGACAGGACAGT,13,GABAergic,Vip Igfbp6,19
92,Vip,AACAACCTCGTTGCCT,21,GABAergic,Vip C1ql1,19
107,Vip,AACAGGGGTGCCGTAC,21,GABAergic,Vip C1ql1,19
...,...,...,...,...,...,...
6270,Vip,TTGCTGCGTACTGCCG,18,GABAergic,Vip Cbln4_1,19
6315,Vip,TTGTTGTAGAGCATTA,21,GABAergic,Vip C1ql1,19
6323,Vip,TTTACCAAGCTGTTCA,11,GABAergic,Vip Htr1f_1,19
6324,Vip,TTTACCAGTAAGTCAA,17,GABAergic,Vip Lypd1,19


In [335]:
meta = pd.DataFrame()
meta['cell_type'] = ds.ca['Cell Type']
meta['cell_barcode'] = ds.ca['barcode']
meta['cluster_id'] = ds.ca['cluster_id']
meta['class_label'] = ds.ca['class_label']
meta['sub_label'] = ds.ca['cluster_label']

uniqs = np.unique(meta['cell_type'])
cell_type_dict = dict(zip(uniqs,range(len(uniqs))))
meta['cell_type_id'] = [cell_type_dict[i] for i in meta['cell_type']]

print(meta.head())
#Just in case
counts = ds.layers['counts'][:,:] #U,S x cells for 2000 hvgs, Raw counts

  cell_type      cell_barcode  cluster_id    class_label     sub_label  \
0     L5 IT  AAACCCAAGTGAACAT          42  Glutamatergic  L5 IT Tcap_2   
1   L2/3 IT  AAACCCAAGTTGAATG          40  Glutamatergic       L2/3 IT   
2     L5 IT  AAACCCACACGATAGG          42  Glutamatergic  L5 IT Tcap_2   
3   L2/3 IT  AAACCCACAGAGGGTT          40  Glutamatergic       L2/3 IT   
4     L6 CT  AAACCCAGTACTAAGA          60  Glutamatergic  L6 CT Cpa6_1   

   cell_type_id  
0             4  
1             2  
2             4  
3             2  
4             6  


In [66]:
#Save with standard names
all_counts = counts.T
allS = all_counts[:,2000:]
allU = all_counts[:,0:2000]
retAdata = anndata.AnnData(
			X=allS,
			layers={
				'spliced': allS,
				'unspliced': allU
			},
			obs=pd.DataFrame({'barcode': ds.ca['barcode']},index=ds.ca['barcode']),
			var=pd.DataFrame({'gene_name': ds.ra['gene_name'][0:2000]},index=ds.ra['gene_name'][0:2000])
		)

retAdata.write_loom('/home/tchari/counts/allen_bivi/loom/correct_B08_processed_hv.loom')

/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [336]:
ds.close()

In [9]:
meta['cell_type'] = [i.replace(' ','_') for i in meta['cell_type']]
meta['cell_type'] = [i.replace('/','-') for i in meta['cell_type']]

In [10]:
meta['cell_type'].value_counts()

L5_IT         2395
L6_CT         1333
L2-3_IT        823
L6_IT          489
Sst            271
L5-6_NP        251
Vip            233
Lamp5          192
Pvalb          114
Oligo           82
L6b             66
Astro           49
Sncg            45
Endo            21
Macrophage      17
OPC             17
L5_ET            9
VLMC             7
L6_IT_Car3       3
SMC              1
Name: cell_type, dtype: int64

In [607]:
cell_type_dict

{'Astro': 0,
 'Endo': 1,
 'L2/3 IT': 2,
 'L5 ET': 3,
 'L5 IT': 4,
 'L5/6 NP': 5,
 'L6 CT': 6,
 'L6 IT': 7,
 'L6 IT Car3': 8,
 'L6b': 9,
 'Lamp5': 10,
 'Macrophage': 11,
 'OPC': 12,
 'Oligo': 13,
 'Pvalb': 14,
 'SMC': 15,
 'Sncg': 16,
 'Sst': 17,
 'VLMC': 18,
 'Vip': 19}